In [4]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# Sentences we want sentence embeddings for
sentences = ['This is an example sentence', 'Each sentence is converted']

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

# Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

# Perform pooling
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

# Normalize embeddings
sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

print("Sentence embeddings:")
print(sentence_embeddings)


Sentence embeddings:
tensor([[ 6.7657e-02,  6.3496e-02,  4.8713e-02,  7.9305e-02,  3.7448e-02,
          2.6527e-03,  3.9375e-02, -7.0984e-03,  5.9361e-02,  3.1537e-02,
          6.0098e-02, -5.2905e-02,  4.0607e-02, -2.5931e-02,  2.9843e-02,
          1.1269e-03,  7.3515e-02, -5.0382e-02, -1.2239e-01,  2.3703e-02,
          2.9727e-02,  4.2477e-02,  2.5634e-02,  1.9952e-03, -5.6919e-02,
         -2.7160e-02, -3.2904e-02,  6.6025e-02,  1.1901e-01, -4.5879e-02,
         -7.2621e-02, -3.2584e-02,  5.2341e-02,  4.5055e-02,  8.2530e-03,
          3.6702e-02, -1.3942e-02,  6.5392e-02, -2.6427e-02,  2.0638e-04,
         -1.3664e-02, -3.6281e-02, -1.9504e-02, -2.8974e-02,  3.9427e-02,
         -8.8409e-02,  2.6242e-03,  1.3671e-02,  4.8306e-02, -3.1157e-02,
         -1.1733e-01, -5.1169e-02, -8.8529e-02, -2.1896e-02,  1.4299e-02,
          4.4417e-02, -1.3481e-02,  7.4339e-02,  2.6638e-02, -1.9876e-02,
          1.7919e-02, -1.0605e-02, -9.0426e-02,  2.1327e-02,  1.4120e-01,
         -6.4717e

In [5]:
# Create model with HuggingFace Hub
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain_classic.chains import RetrievalQA
from langchain_core.prompts import PromptTemplate
import os

# Load .env variables for HuggingFace
from dotenv import load_dotenv
# Set env path for notebook environment without __file__
env_path  = os.path.join(os.getcwd(), '.env')
load_dotenv(dotenv_path=env_path)

# =========================
# STEP 1: LOAD PDF DOCUMENT
# =========================
loader = PyPDFLoader("dataset/Pdf/Standard_container_operations.pdf")
documents = loader.load()
print(f"Loaded {len(documents)} documents from the PDF.")

# =========================
# STEP 2: CREATE VECTOR STORE
# =========================
print("Creating vector store...")
embedding_model = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=embedding_model)

vectorstore = FAISS.from_documents(documents, embeddings)
print(f"Vector store created with {vectorstore.index.ntotal} vectors.")

# =========================
# STEP 3: SET UP HUGGINGFACE LLM
# =========================
token_hf = os.getenv("HUGGING_API_KEY")

# Initialize HuggingFace LLM
llm_base = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.2",
    task="conversational", # Set this explicitly to match the provider
    temperature=0.3,
    max_new_tokens=512,
    top_p=0.50,
    huggingfacehub_api_token=token_hf

)

# Wrap in ChatHuggingFace for chat-based interaction
llm = ChatHuggingFace(llm=llm_base)

# =========================
# STEP 4: CREATE PROMPT TEMPLATE
# =========================
prompt_template = """You are an expert Inspector Engineer in shipment and container operations.

Use the following context from the Standard Container Operations document to answer the question.
If you don't know the answer based on the context, say "I don't have enough information in the document to answer that."

Context:
{context}

Question: {question}

Answer (provide detailed, technical response):"""

PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

# =========================
# STEP 5: CREATE QA CHAIN
# =========================
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_kwargs={"k": 3}),
    chain_type_kwargs={"prompt": PROMPT},
    return_source_documents=True
)

# =========================
# STEP 6: ASK QUESTIONS
# =========================
questions = [
    "What are the damage types mentioned in the document?",
    "What are the damage constructions mentioned in the document?",
    "Explain the inspection process for container operations simply."
]

print("\n--- QA Results ---\n")
print("AI Inspector Engineer Responses:\n")

for i, question in enumerate(questions, 1):
    print(f"Question {i}: {question}")

    result = qa_chain.invoke({"query": question})

    print("Answer:")
    print(result['result'])

    print("\nSource Documents:")
    for j, doc in enumerate(result['source_documents'], 1):
        print(f"\n Source Document {j} (page {doc.metadata.get('page', 'N/A')}):\n")
        print(f"{doc.page_content[:200]}...")  # Print first 200 characters of the source document

Loaded 75 documents from the PDF.
Creating vector store...
Vector store created with 75 vectors.

--- QA Results ---

AI Inspector Engineer Responses:

Question 1: What are the damage types mentioned in the document?
Answer:
 Based on the context provided in the document, the following damage types are mentioned:

1. Dents: Dents are acceptable as per Marine Order 44, CSC, and ISO Standards.
2. Previous repair: A previous repair is acceptable as long as it is sound.
3. Light rust: Light rust occurring is acceptable as long as there is no blistering.
4. Hole in wall: A hole in the wall has been repaired via the insertion of a welded panel that has been sealed and painted.
5. Blistering corrosion: Assessment is required for blistering corrosion to assess the structural integrity of the container.
6. Small cracks: Assessment is required for damage to the exterior wall to ascertain that no small cracks have penetrated to the inside of the container.
7. Flooring damage: Flooring damage with